In [147]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn

In [148]:
data = pd.read_csv("D:\CODE\jupyter notebook\data\(日月光华)PyTorch深度学习教程课件\(日月光华)PyTorch深度学习教程课件\daatset\HR.csv")
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   part                   14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [150]:
data.salary.unique()

array(['low', 'medium', 'high'], dtype=object)

In [151]:
data.groupby(['salary', 'part']).size()

salary  part       
high    IT               83
        RandD            51
        accounting       74
        hr               45
        management      225
        marketing        80
        product_mng      68
        sales           269
        support         141
        technical       201
low     IT              609
        RandD           364
        accounting      358
        hr              335
        management      180
        marketing       402
        product_mng     451
        sales          2099
        support        1146
        technical      1372
medium  IT              535
        RandD           372
        accounting      335
        hr              359
        management      225
        marketing       376
        product_mng     383
        sales          1772
        support         942
        technical      1147
dtype: int64

In [152]:
# 转化为one_hot编码
data = data.join(pd.get_dummies(data.salary))
# 删除那一列
del data["salary"]
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,part,high,low,medium
0,0.38,0.53,2,157,3,0,1,0,sales,False,True,False
1,0.80,0.86,5,262,6,0,1,0,sales,False,False,True
2,0.11,0.88,7,272,4,0,1,0,sales,False,False,True
3,0.72,0.87,5,223,5,0,1,0,sales,False,True,False
4,0.37,0.52,2,159,3,0,1,0,sales,False,True,False


In [153]:
data = data.join(pd.get_dummies(data.part))
del data["part"]
data = data * 1
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,high,low,...,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.38,0.53,2,157,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.80,0.86,5,262,6,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.11,0.88,7,272,4,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.72,0.87,5,223,5,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.37,0.52,2,159,3,0,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [154]:
data.shape

(14999, 21)

In [155]:
Y = torch.from_numpy(data.left.values.reshape(-1, 1)).type(torch.FloatTensor)
Y.shape

torch.Size([14999, 1])

In [156]:
X_data = data[[c for c in data.columns if c!='left']].values


In [157]:
X = torch.from_numpy(X_data).type(torch.FloatTensor)

In [158]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(20, 64)
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        y_pred = self.linear1(input)
        y_pred = self.relu(y_pred)
        y_pred = self.linear2(y_pred)
        y_pred = self.relu(y_pred)
        y_pred = self.linear3(y_pred)
        y_pred = self.sigmoid(y_pred)
        return y_pred

In [159]:
model = Model()
opt = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_fn = nn.BCELoss()
batch_size = 64
num_batch = X.size(0)//batch_size
epoches = 100
model

Model(
  (linear1): Linear(in_features=20, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [160]:
for epoch in range(epoches):
    for i in range(num_batch):
        start = i * batch_size
        end = start + batch_size
        x = X[start:end]
        y = Y[start:end]
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        print("epoch: ", epoch, "      ", "loss: ", loss_fn(model(X), Y).data.item())

epoch:  0        loss:  0.6736930012702942
epoch:  1        loss:  0.6861065030097961
epoch:  2        loss:  0.6867752075195312
epoch:  3        loss:  0.6814599633216858
epoch:  4        loss:  0.6759210228919983
epoch:  5        loss:  0.6689117550849915
epoch:  6        loss:  0.6619674563407898
epoch:  7        loss:  0.6786454916000366
epoch:  8        loss:  0.6379754543304443
epoch:  9        loss:  0.6302241086959839
epoch:  10        loss:  0.6225878000259399
epoch:  11        loss:  0.6151961088180542
epoch:  12        loss:  0.6079743504524231
epoch:  13        loss:  0.6012352705001831
epoch:  14        loss:  0.5947663187980652
epoch:  15        loss:  0.588639497756958
epoch:  16        loss:  0.5874483585357666
epoch:  17        loss:  0.5795996785163879
epoch:  18        loss:  0.5744980573654175
epoch:  19        loss:  0.5678955316543579
epoch:  20        loss:  0.5646229386329651
epoch:  21        loss:  0.5792536735534668
epoch:  22        loss:  0.569352388381958


# 使用dataset, dataloader类加载数据

In [179]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [180]:
dataset = TensorDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [170]:
for epoch in range(epoches):
    for x, y in dataloader:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        print("epoch: ", epoch, "      ", "loss: ", loss_fn(model(X), Y).detach())

epoch:  0        loss:  tensor(0.2156)
epoch:  1        loss:  tensor(0.2173)
epoch:  2        loss:  tensor(0.2176)
epoch:  3        loss:  tensor(0.2178)
epoch:  4        loss:  tensor(0.2153)
epoch:  5        loss:  tensor(0.2144)
epoch:  6        loss:  tensor(0.2172)
epoch:  7        loss:  tensor(0.2146)
epoch:  8        loss:  tensor(0.2152)
epoch:  9        loss:  tensor(0.2141)
epoch:  10        loss:  tensor(0.2220)
epoch:  11        loss:  tensor(0.2136)
epoch:  12        loss:  tensor(0.2161)
epoch:  13        loss:  tensor(0.2135)
epoch:  14        loss:  tensor(0.2165)
epoch:  15        loss:  tensor(0.2136)
epoch:  16        loss:  tensor(0.2145)
epoch:  17        loss:  tensor(0.2138)
epoch:  18        loss:  tensor(0.2136)
epoch:  19        loss:  tensor(0.2123)
epoch:  20        loss:  tensor(0.2151)
epoch:  21        loss:  tensor(0.2129)
epoch:  22        loss:  tensor(0.2178)
epoch:  23        loss:  tensor(0.2128)
epoch:  24        loss:  tensor(0.2133)
epoch:  25

# 添加验证集

In [171]:
from sklearn.model_selection import train_test_split

In [182]:
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data,test_size=0.2)
train_x = torch.from_numpy(train_x).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
test_x = torch.from_numpy(test_x).type(torch.float32)
test_y = torch.from_numpy(test_y).type(torch.float32)

In [187]:
train_dataset = TensorDataset(train_x, train_y)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = TensorDataset(test_x, test_y)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [186]:
def accuracy(y_pred, y):
    y_pred = (y_pred > 0.5).type(torch.int32)
    ac = (y_pred==y).float().mean()
    return ac

In [192]:
model = Model()
opt = torch.optim.Adam(model.parameters(), lr = 0.0001)
loss_fn = nn.BCELoss()
batch_size = 64
num_batch = X.size(0)//batch_size
epoches = 100
for epoch in range(epoches):
    for train_x, train_y in train_dataloader:
        train_y_pred = model(train_x)
        loss = loss_fn(train_y_pred, train_y)
        opt.zero_grad()
        loss.backward()
        opt.step()
    with torch.no_grad():
        print("epoch: ", epoch, "      ", "loss: ", loss_fn(model(X), Y).detach(), "      ", "accuracy: ", accuracy(model(test_x), test_y))
        

epoch:  0        loss:  tensor(0.5640)        accuracy:  tensor(0.7543)
epoch:  1        loss:  tensor(0.5598)        accuracy:  tensor(0.7543)
epoch:  2        loss:  tensor(0.5557)        accuracy:  tensor(0.7543)
epoch:  3        loss:  tensor(0.5500)        accuracy:  tensor(0.7543)
epoch:  4        loss:  tensor(0.5448)        accuracy:  tensor(0.7543)
epoch:  5        loss:  tensor(0.5369)        accuracy:  tensor(0.7543)
epoch:  6        loss:  tensor(0.5300)        accuracy:  tensor(0.7543)
epoch:  7        loss:  tensor(0.5227)        accuracy:  tensor(0.7543)
epoch:  8        loss:  tensor(0.5158)        accuracy:  tensor(0.7543)
epoch:  9        loss:  tensor(0.5171)        accuracy:  tensor(0.7543)
epoch:  10        loss:  tensor(0.4984)        accuracy:  tensor(0.7543)
epoch:  11        loss:  tensor(0.4906)        accuracy:  tensor(0.7543)
epoch:  12        loss:  tensor(0.4837)        accuracy:  tensor(0.7543)
epoch:  13        loss:  tensor(0.4725)        accuracy:  ten